In [ ]:
### time series forecasting 
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
from numpy.typing import ArrayLike

from src import config

pd.set_option("display.max_columns", None)

%matplotlib inline
sns.set_style('white')
sns.set_palette('deep')

In [ ]:
!pip install statsmodels
!pip install xgboost

In [ ]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from statsmodels.tsa.seasonal import seasonal_decompose

In [ ]:
# read in data 
df = pd.read_parquet(config.INT_FILE_PATH / 'transactions.parquet')

In [ ]:
df.head()

In [ ]:
df = df[["order_purchase_timestamp", "order_total_price"]]

In [ ]:
weekly_sales = df.set_index("order_purchase_timestamp").resample("W")[["order_total_price"]].sum()

In [ ]:
weekly_sales

In [ ]:
decompose = seasonal_decompose(weekly_sales, extrapolate_trend=12)

In [ ]:
# Observed
obs = decompose.observed
# Trend
trend = decompose.trend
# Seasonal
season = decompose.seasonal
# Error
random = decompose.resid


In [ ]:
fig, axes = plt.subplots(4, 1, figsize=(15,8), sharex=True)
fig.suptitle('Time Series of Purchase Values')

sns.lineplot(x=obs.index, y=obs, ax=axes[0], data=obs)
sns.lineplot(x=trend.index, y=trend, ax=axes[1], data=trend)
sns.lineplot(x=season.index, y=season, ax=axes[2], data=season)
sns.lineplot(x=random.index, y=random, ax=axes[3], data=random)

plt.show()

In [ ]:
weekly_sales.info()

In [ ]:
weekly_sales_filt = weekly_sales.loc["2017-01-08":'2018-09-02',]

In [ ]:
weekly_sales_filt["X_1"] = weekly_sales_filt["order_total_price"].shift(1)
weekly_sales_filt["X_2"] = weekly_sales_filt["order_total_price"].shift(2)
weekly_sales_filt["X_3"] = weekly_sales_filt["order_total_price"].shift(3)

In [ ]:
train_size = int(len(weekly_sales_filt) * 0.80)
train, test = weekly_sales_filt[:train_size], weekly_sales_filt[train_size:]

In [ ]:
train.head()

In [ ]:
X_train, y_train = train.drop(columns="order_total_price"), train["order_total_price"]
X_test, y_test = test.drop(columns="order_total_price"), test["order_total_price"]

## modelling 

In [ ]:
## create function to define MAPE
def mean_absolute_percentage_error(actual: ArrayLike, pred: ArrayLike): 
    actual, pred = np.array(actual), np.array(pred)
    return np.mean(np.abs((actual - pred) / actual))

In [ ]:
xgb_model = xgb.XGBRegressor(n_estimators=400, learning_rate=0.03)
xgb_model.fit(X_train, y_train, 
        early_stopping_rounds=5, 
        eval_set=[(X_test, y_test)], 
        verbose=False)

y_pred = xgb_model.predict(X_test)

print(f"RMSE for xgb is: {mean_squared_error(y_test, y_pred, squared=False)}")
print(f"MAPE for xgb is {mean_absolute_percentage_error(y_test, y_pred)}")

In [ ]:
train_pred = pd.Series(xgb_model.predict(X_train))
test_pred = pd.Series(xgb_model.predict(X_test))

predictions = pd.concat([train_pred, test_pred], axis=0)
predictions.index = weekly_sales_filt.index

ax = plt.gca()

weekly_sales_filt[['order_total_price']].plot(figsize=(15, 6), ax=ax)
predictions.plot(figsize=(15, 6), ax=ax, color="orange")
plt.show()